In [ ]:
# Imports
import os
import asyncio
from typing import cast, List, Tuple
from dotenv import load_dotenv

# Agent Framwork
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI, set_tracing_disabled
from opentelemetry import trace as otel_trace
from phoenix.otel import register

# Markdown output display
from IPython.display import Markdown, display

In [132]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [133]:
# Environment Variables

# Load Environment Variables
load_dotenv(override=True)

# Ollama Environment Variables
ollama_api_key = env_to_str('OLLAMA_API_KEY')
ollama_base_url = env_to_str('OLLAMA_BASE_URL')

# Gemini Environment Variables
gemini_api_key = env_to_str("GEMINI_API_KEY")
gemini_base_url = env_to_str("GEMINI_BASE_URL")

# Phoenix Collector Variables
phoenix_collector_endpoint = env_to_str("PHOENIX_COLLECTOR_ENDPOINT") or "http://localhost:6006/v1/traces"

# Maileroo Environment Variables
maileroo_api_key = env_to_str("MAILEROO_API_KEY")
maileroo_base_url = env_to_str("MAILEROO_BASE_URL")
maileroo_template_url = env_to_str("MAILEROO_TEMPLATE_URL")

In [134]:
# Available AI Models

# Ollama General Models
model_mistral = env_to_str("MODEL_MISTRAL")
model_llama = env_to_str('MODEL_LLAMA3')
model_mistral_nemo = env_to_str("MODEL_MISTRAL_NEMO")
model_gemma = env_to_str("MODEL_GEMMA3")

# Ollama Resoning Models 
model_phi = env_to_str("MODEL_PHI3.5")
model_qwen = env_to_str("MODEL_QWEN3")
model_deepseek = env_to_str("MODEL_DEEPSEEK_R1")

# Gemini Models
model_gemini_flash = env_to_str("MODEL_GEMINI_FLASH")

In [135]:
# Attribute Access Class
class AttributeAccess:
    
    # Constructor
    def __init__(self, dictionary: dict):
        
        # Attribute Access
        for key, value in dictionary.items():
            if isinstance(value, dict):
                value = AttributeAccess(value)
            
            # Dot Notation Setter
            setattr(self, key, value)

    # Bracket Notation Getter
    def __getitem__(self, key):
        return getattr(self, key)
    
    # Bracket Notation Setter
    def __setitem__(self, key, value):
        setattr(self, key, value)
        
	# Convert Back to Python dict
    def to_dict(self) -> dict:
        result_dict = {}

        for key, value in self.__dict__.items():
            if isinstance(value, AttributeAccess):
                result_dict[key] = value.to_dict()
            else:
                result_dict[key] = value
        return result_dict

In [136]:
# Configurations

# Set Current Model
selected_model = model_llama

# Multi Model Flag
multi_model = False

# Check the need of multiple models, 
# if not, select current ai model.
if multi_model and selected_model != "":
    print("This is a multi model setup. Leave current model as empty string: ''.")
elif multi_model and selected_model == "":
    print("This is a multi model setup. ")
elif not multi_model and selected_model == "":
    print("This is a single model setup. Please set a current model.")
elif not multi_model and selected_model != "":
    print(f"This is a single model setup. The current model is: {model_llama}")

This is a single model setup. The current model is: llama3.1:8b


In [ ]:
# Configure Phoenix Tracer

set_tracing_disabled(True)

if selected_model != "":
    phoenix_project_name = f"{selected_model}_phoenix"
else:
    phoenix_project_name = "multi_model_phoenix"

try:
    tracer_provider = register(
    project_name=phoenix_project_name,
    auto_instrument=True,
    endpoint=phoenix_collector_endpoint,
    set_global_tracer_provider=True
)
    print("Phoenix tracer registered successfully")
except Exception as e:
    print(f"Phoenix registration failed: {e}")

Overriding of current TracerProvider is not allowed
DependencyConflict: requested: "openai-agents >= 0.1.0" but found: "openai-agents 0.0.17"
DependencyConflict: requested: "google-genai" but found: "None"


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: llama3.1:8b_phoenix
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

Phoenix tracer registered successfully


In [138]:
# Llama3 Model
llama = OpenAIChatCompletionsModel(
    model=selected_model,
    openai_client=AsyncOpenAI(
        api_key=ollama_api_key, 
        base_url=ollama_base_url
    )
)

In [139]:
# Agent Workflow Instructions
data_a = {
    "name":"Professional Sales Agent",
    "instructions":"You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails.",
    "model":llama
}
agent_a = AttributeAccess(data_a)

data_b = {
    "name":"Engaging Sales Agent",
    "instructions":"You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response.",
    "model":llama
}
agent_b = AttributeAccess(data_b)

data_c = {
    "name":"Busy Sales Agent",
    "instructions":"You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails.",
    "model":llama
}
agent_c = AttributeAccess(data_c)

data_d = {
    "name":"sales_picker",
    "instructions":"You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    "model":llama
}
agent_d = AttributeAccess(data_d)

In [140]:
# Agents

sales_agent_a = Agent(
    name=agent_a.name,
    instructions=agent_a.instructions,
    model=agent_a.model
)

sales_agent_b = Agent(
    name=agent_b.name,
    instructions=agent_b.instructions,
    model=agent_b.model
)

sales_agent_c = Agent(
    name=agent_c.name,
    instructions=agent_c.instructions,
    model=agent_c.model
)

sales_picker = Agent(
    name=agent_d.name,
    instructions=agent_d.instructions,
    model=agent_d.model
)

In [141]:
# Agentic Workflow

class AgenticWorkflow:
    # Constructor
    def __init__(
            self, 
            tracer_name: str, 
            model_name: str, 
            generation_agents: List[Agent], 
            evaluator_agent: Agent):
        self.tracer_name = tracer_name
        self.model_name = model_name
        self.generation_agents = generation_agents
        self.evaluator_agent = evaluator_agent

        # Arize Phoenix 
        # OpenTelemetry Tracer
        self.tracer = otel_trace.get_tracer(__name__)

    # Run Generation
    async def _run_generation(self, prompt: str) -> List[str]:
        with self.tracer.start_as_current_span(f"{self.tracer_name} - Generation") as span:
            span.set_attribute("phase", "generation")
            span.set_attribute("input.prompt", prompt)
            span.set_attribute("model.used", self.model_name)

            # Check for generation agents provided
            if not self.generation_agents:
                print(f"Warning: No generation agents provided for '{self.tracer_name}'.")
                return []
            
            # Run all generation agents
            agent_run_results = await asyncio.gather(
                *[Runner.run(agent, prompt) for agent in self.generation_agents]
            )

            # Extract Final Outputs
            generated_outputs = [result.final_output for result in agent_run_results]

            if generated_outputs:
                total_length = sum(len(output) for output in generated_outputs)
                span.set_attribute("generation.count", len(generated_outputs))
                span.set_attribute("generation.total_length", total_length)
                span.set_attribute("generation.first_preview", generated_outputs[0][:100] if generated_outputs[0] else "")

            return generated_outputs
    
    # Run Evaluation
    async def _run_evaluation(self, generated_content: List[str]) -> str:
            with self.tracer.start_as_current_span(f"{self.tracer_name} - Evaluation") as span:
                span.set_attribute("phase", "evaluation")
                span.set_attribute("evaluator.agent_name", self.evaluator_agent.name)

                # Combine generated content into a string
                evaluator_input = "\n\n--- Option ---\n\n".join(generated_content)

                span.set_attribute("evaluator.input_preview", evaluator_input[:500] if evaluator_input else "")

            # Run Evaluator Agent
            eval_result = await Runner.run(self.evaluator_agent, evaluator_input)
            final_output = eval_result.final_output

            span.set_attribute("evaluation.output_length", len(final_output))
            span.set_attribute("evaluation.output_preview", final_output[:100] if final_output else "")

            return final_output
    
    # Execute Workflow
    async def execute_workflow(self, initial_prompt: str) -> Tuple[List[str], str]:
        with self.tracer.start_as_current_span(self.tracer_name) as main_span:
            main_span.set_attribute("workflow.type", "generic_generation_evaluation")
            main_span.set_attribute("workflow.initial_prompt", initial_prompt)

            # Execute Generation
            generated_items = await self._run_generation(initial_prompt)

            # Execute Evaluation
            evaluated_item = await self._run_evaluation(generated_items)

            print(f"\n--- All Generated Items for '{self.tracer_name}' ---")
            if generated_items:
                for i, item in enumerate(generated_items):
                    display(Markdown(f"## Generated Item {i+1}:\n\n{item}\n\n---"))
            else:
                print("No items were generated.")

            print(f"\n--- Best Evaluated Item for '{self.tracer_name}' ---")
            if evaluated_item:
                display(Markdown(evaluated_item))
            else:
                print("No item was evaluated/picked.")


            # Final Summary Attributes
            main_span.set_attribute("final.generated_count", len(generated_items))
            main_span.set_attribute("final.evaluated_item_length", len(evaluated_item))
            main_span.set_attribute("final.evaluated_item_preview", evaluated_item[:100] if evaluated_item else "")

        return generated_items, evaluated_item

In [142]:
# Workflow Variables
tracer_name = "Parallel Cold Emails"
model_name = model_llama
generation_agents = [sales_agent_a, sales_agent_b, sales_agent_c]
evaluator_agent = sales_picker

# Init Workflow
sales_email = AgenticWorkflow(
    tracer_name, 
    model_name, 
    generation_agents, 
    evaluator_agent
)

# Run Workflow
all_emails, best_email = await sales_email.execute_workflow("Write a cold sales email.")

[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: ollama_o*********ault. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: ollama_o*********ault. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: ollama_o*********ault. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
Setting attribute on ended span.
Setting attribute on ended span.



--- All Generated Items for 'Parallel Cold Emails' ---


## Generated Item 1:

Subject: Ensure Your Organization's Compliance with Our Expertise

Dear [Recipient's Name],

I am reaching out from ComplAI, a company specializing in SOC2 compliance solutions powered by artificial intelligence. As an IT risk management and security professional, I came across your organization while researching companies actively seeking to maintain robust cybersecurity standards.

Our AI-driven SaaS platform has aided numerous organizations in streamlining their audit preparations and ensuring SOC2 compliance. It features automated workflows for policy documentation, controls assessment, and report analysis, providing transparency into your security posture.

Some key benefits of our solution include:

1. **Reduced audit risk**: By pre-emptively identifying areas for improvement, you'll avoid costly and time-consuming auditors.
2. **Operational efficiency**: Our AI tool streamlines the compliance process, allowing your team to focus on business operations instead of laborious paperwork.
3. **Improved board satisfaction**: With ComplAI's tools at hand, management can be confident in your organization's adherence to regulatory standards.

Considering your organization's commitment to maintaining strong IT and security practices, we would like to schedule a call to discuss the features and benefits our solution provides. If you're interested or prefer email exchange for initial questions, please reply to this message. I have attached some case studies for your reference as well.

Thank you for taking the time to consider how ComplAI might be able to assist you in strengthening your organization's security posture.

Sincerely,
[Your Name]
ComplAI Sales Team
[Company Email Address]

---

## Generated Item 2:

Here's a shot at it:

Subject: Don't Let SOC 2 Send Your Sanity Packing 

Hi [First Name],

Are you feeling like you're stuck in a compliance nightmare? Like, "I've been up all night calculating the 'right' way to count those pesky customer identity attributes"?

Well, buckle up friend! As someone who's worked with numerous companies like yours, I can confidently say: ComplAI is here to save your sanity (and your SOC 2 audit).

Our AI-powered tool makes compliance a breeze. We'll guide you through the complex standards, provide you with customizable controls, and even generate reports that'll make your auditor proud.

Ready to trade late nights for late coffee breaks? Let's chat about how ComplAI can help you conquer SOC 2 like a pro:

[Your Contact Info]

Best,
[Your Name]
ComplAI 

P.S. Don't let compliance drain the fun out of innovation. There are better uses for your time than wrestling with policy matrices.

This email aims to:

1. Grab attention with a playful subject line and opening sentence.
2. Identify the pain point (compliance dread).
3. Introduce ComplAI as a solution that alleviates this pain.
4. End on an engaging note, inviting the recipient to discuss further.

Keep in mind: Sales emails are most effective when personalized and tailored to the recipient's specific needs. This is just a starting point; you'd likely want to customize it based on your research or interactions with the target company!

---

## Generated Item 3:

Here's a sample cold email:

Subject: Streamline Your Audit Preparation with ComplAI

Dear [First Name],

I came across your company and noticed you're likely handling sensitive customer data. Ensuring SOC2 compliance can be complex, but it's essential for maintaining trust with stakeholders.

ComplAI offers an AI-powered SaaS tool to simplify your audit preparation. Our platform automates documentation, risk assessments, and gap analysis, saving your team time and effort.

Would you like to discuss how ComplAI can support your SOC2 compliance needs?

Best,
[Your Name]

This cold email aims to:

1. Set the context for why SOC2 compliance is relevant (sensitive customer data).
2. Introduce ComplAI as a solution.
3. Offer value by highlighting the benefits of using the platform (time-saving and effort reduction).

Keep it concise and free of jargon to increase the chances of getting noticed!

---


--- Best Evaluated Item for 'Parallel Cold Emails' ---


Subject: Don't Let SOC 2 Send Your Sanity Packing 

Hi [First Name],

Are you feeling like you're stuck in a compliance nightmare? Like, "I've been up all night calculating the 'right' way to count those pesky customer identity attributes"?

Well, buckle up friend! As someone who's worked with numerous companies like yours, I can confidently say: ComplAI is here to save your sanity (and your SOC 2 audit).

Our AI-powered tool makes compliance a breeze. We'll guide you through the complex standards, provide you with customizable controls, and even generate reports that'll make your auditor proud.

Ready to trade late nights for late coffee breaks? Let's chat about how ComplAI can help you conquer SOC 2 like a pro:

[Your Contact Info]

Best,
[Your Name]
ComplAI 

P.S. Don't let compliance drain the fun out of innovation. There are better uses for your time than wrestling with policy matrices.

[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: ollama_o*********ault. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
